## MDLE - Project 2

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

tracks = spark.read.format("csv") \
    .option("header", "true") \
    .load("fma_metadata/tracks.csv")

tracks.show()



'''small_tracks = tracks.where((tracks.subset == 'small'))
small_tracks.show()
print(small_tracks.select('set31'))'''

23/04/19 12:08:12 WARN Utils: Your hostname, Pedros-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/04/19 12:08:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/19 12:08:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/19 12:08:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/04/19 12:08:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, 

+--------------------+--------------------+-------------------+--------------------+--------------+---------+---------+--------------------+--------+----------------+----------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+-------------------+-------------------+--------+---------+---------+--------------------+----------+-----------+--------------------+-------------------+--------------------+--------+--------+-------+---------+---------+-------------------+
|                 _c0|              album1|             album2|              album3|        album4|   album5|   

"small_tracks = tracks.where((tracks.subset == 'small'))\nsmall_tracks.show()\nprint(small_tracks.select('set31'))"

In [4]:
from pyspark.sql.functions import col


first_two_rows = tracks.take(2)
first_row = first_two_rows[0]
second_row = first_two_rows[1]
#new_column_names = [str(cell) for cell in first_row]
new_column_names = [str(cell) for cell in first_row[1:]] + [str(second_row[0])]
renamed_tracks = tracks.toDF(*new_column_names)





23/04/19 12:08:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [5]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id
data_with_id = renamed_tracks.withColumn("unique_id", monotonically_increasing_id())
min_id = data_with_id.select("unique_id").orderBy("unique_id").first()[0]
second_id = data_with_id.select("unique_id").orderBy("unique_id").take(2)[-1][0]

data_without_first_row = data_with_id.filter(col("unique_id") != min_id).drop("unique_id")
data_without_second_row = data_without_first_row.filter(col("unique_id") != second_id).drop("unique_id")

data_without_second_row.show()


+--------------------+--------------------+-------------------+--------------------+--------------+--------+-----------+--------------------+--------+----------------+----------+--------------------+--------------------+-----------------+-------------------+-------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+-------------------+-------------------+---------+---------+-------+--------------------+-----------+--------+--------------------+-------------------+--------------------+--------+------+---------+----+---------+-------------------+
|            comments|        date_created|      date_released|            engineer|     favorites|      id|

23/04/19 12:08:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [6]:
col_list = data_without_second_row.columns
shifted_col_list = [col_list[-1]] + col_list[:-1]
shifted_df = data_without_second_row.toDF(*shifted_col_list)
shifted_df.show()

+--------------------+--------------------+-------------------+--------------------+--------------+---------+---------+--------------------+--------+----------------+----------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+-------------------+-------------------+--------+---------+---------+--------------------+----------+-----------+--------------------+-------------------+--------------------+--------+--------+------+---------+---------+-------------------+
|            track_id|            comments|       date_created|       date_released|      engineer|favorites|    

23/04/19 12:08:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [7]:
shifted_df.where(shifted_df.subset == 'small').show()

23/04/19 12:08:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

+--------+--------+-------------------+-------------------+-----------------+---------+----+--------------------+-------+------------+-------------+--------------------+------+----------------+-------------------+---------------+--------------------+--------------------+--------+-------------------+---------+----+----------+----------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+------+--------+--------+-------------+-------------------+-------------------+--------+---------+------------+--------+----------+--------------------+--------+-------------+--------------------+-------+--------+------+---------+-------------+--------------------+
|track_id|comments|       date_created|      date_released|         engineer|favorites|  id|         information|listens|    producer|         tags|               title|tracks|            type|  active_year_begin|active_year_end|   associate

In [7]:
from pyspark.sql.functions import col, when
def merge_columns(df, col_name):
    duplicate_columns = [c for c in df.columns if c == col_name]
    
    if len(duplicate_columns) <= 1:
        return df
    
    merged_col = col(duplicate_columns[0])
    
    for c in duplicate_columns[1:]:
        merged_col = when(merged_col == 0, col(c)).otherwise(merged_col)
    
    df = df.drop(*duplicate_columns).withColumn(col_name, merged_col)
    return df

In [8]:
merged_comments = merge_columns(shifted_df, 'comments')

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `comments` cannot be resolved. Did you mean one of the following? [`composer`, `listens`, `listens`, `members`, `genres`].;
'Project [track_id#1368, date_created#1370, date_released#1371, engineer#1372, favorites#1373, id#1374, information#1375, listens#1376, producer#1377, tags#1378, title#1379, tracks#1380, type#1381, active_year_begin#1382, active_year_end#1383, associated_labels#1384, bio#1385, date_created#1387, favorites#1388, id#1389, latitude#1390, location#1391, longitude#1392, members#1393, ... 27 more fields]
+- Project [track_id#1368, date_created#1370, date_released#1371, engineer#1372, favorites#1373, id#1374, information#1375, listens#1376, producer#1377, tags#1378, title#1379, tracks#1380, type#1381, active_year_begin#1382, active_year_end#1383, associated_labels#1384, bio#1385, date_created#1387, favorites#1388, id#1389, latitude#1390, location#1391, longitude#1392, members#

In [13]:
small_dataset = tracks[tracks['subset'] == 'small']
#small_features = features.loc[small_dataset.index]


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `subset` cannot be resolved. Did you mean one of the following? [`_c0`, `album1`, `album2`, `album3`, `album4`, `album5`, `album6`, `album7`, `album8`, `album9`, `album10`, `album11`, `album12`, `album13`, `artist14`, `artist15`, `artist16`, `artist17`, `artist18`, `artist19`, `artist20`, `artist21`, `artist22`, `artist23`, `artist24`, `artist25`, `artist26`, `artist27`, `artist28`, `artist29`, `artist30`, `set31`, `set32`, `track33`, `track34`, `track35`, `track36`, `track37`, `track38`, `track39`, `track40`, `track41`, `track42`, `track43`, `track44`, `track45`, `track46`, `track47`, `track48`, `track49`, `track50`, `track51`, `track52`].

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import pairwise_distances

# Step 1: Load data
pandas_df = shifted_df.toPandas()  # Convert PySpark DataFrame to Pandas DataFrame
num_features = len(pandas_df.columns)
data = pd.get_dummies(pandas_df).values.astype(np.float32)  # One-hot encode categorical features

# Step 2: Apply agglomerative hierarchical clustering
for k in range(8, 17):
    clustering = AgglomerativeClustering(n_clusters=k, linkage='ward')
    clustering.fit(data)

    # Step 3: Calculate cluster properties
    for i in range(k):
        cluster_indices = np.where(clustering.labels_ == i)[0]
        cluster_data = data[cluster_indices]
        center = np.mean(cluster_data, axis=0)
        distances = pairwise_distances(cluster_data, center.reshape(1, -1))
        radius = np.max(distances)
        diameter = np.max(pairwise_distances(cluster_data))
        density = len(cluster_indices) / ((4/3) * np.pi * radius**3)

        # Step 4: Output results
        print(f'Cluster {i} with {len(cluster_indices)} songs (k={k}):')
        print(f'Radius: {radius:.2f}')
        print(f'Diameter: {diameter:.2f}')
        print(f'Density (using r2): {density:.2f}')
        print(f'Density (using d2): {len(cluster_indices) / (np.pi * diameter**2):.2f}')


23/04/19 12:08:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

: 

: 